In [12]:
import os.path
import time

import yfinance as yf
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import math
import tqdm
import numpy as np
import itertools
import pickle
import os
import pygad

#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

plt.rcParams['figure.figsize'] = [15, 10]

a = pd.read_html("https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average")

In [13]:
test = pd.read_csv("historical_stock_prices.csv",usecols=[0,1,2,6,7],index_col=[0])

testing dataframes

In [21]:
test.loc[:,'open'].sum()

1595236833.0155377

In [9]:
print(test.iloc[0])

open      1.0
close     1.0
volume      1
date        1
Name: AHH, dtype: object


In [ ]:
test['date'] = pd.to_datetime(test['date'])

In [ ]:
valid_stonk = test.loc[test["date"]<"1.1.2000"].groupby("ticker").sum().index

c = test.groupby("ticker").sum()
c = c.loc[valid_stonk].sort_values(by=["volume"],ascending=False).head(100)

In [ ]:
tickerji = c.index

In [ ]:
#za trenirati uporabimo prvih deset let
groupe = test.groupby("ticker")
ticker400_data = {g:groupe.get_group(g) for g in groupe.groups if g in tickerji}

def trendReversals(df,percent,prevPivot = -1,sgn=None):
    #če ni prejšne regerenčne točke postavimo na prvo točko kot referenčno
    if prevPivot == -1:
        prevPivot = 0
    #resetiramo highest in lowest na prejšno referenčno točko, highest bo zvišveal prejšno referenčno točko saj le ta
    #je registrirana ko pridemo percent število procentov sstran od prejšne referenčne točke to ne pomeni da je to najvišja točka pivota
    #le da se je zamenjal trend
    highest = lowest = prevPivot
    #iteriramo skozi podatke v podatkovni monžici
    for i in range(prevPivot,len(df.index)):
        #če je trenutna točka višja od točke shranjene v highest
        if df.iloc[i]['open'] > df.iloc[highest]['open']:
            #v primeru da je bil prejšnji trend pozitiven in je definiran zvišujemo tudi lokacijo pivota saj je le ta najvišja točka pred spremembo trenda
            if sgn is not None and sgn:
                prevPivot = i
            #posodobimo vrednost highest z novo najvišjo vrednostjo
            highest = i
        #če je trenutna točka nižja od točke shranjene v lowest
        if df.iloc[i]['open'] < df.iloc[lowest]['open']:
            #če je prejšnji trend definiran in je negativen torej je pivot točka najnižja točka prejšnjega trenda in jo vsakič updatamo ko najdemo nižjo
            if sgn is not None and not sgn:
                prevPivot = i
            #posodobimo najnižjo točko vsakič ko je trenutna točka nižja
            lowest = i
        # če je prejšnji trend nedefiniran ali negativen in je trneutna točka od prejšnjega pivota(najvišje/nižje točke trenda) oddaljena za percent
        # se je trend zamenjal in sedaj iščemo novo pivot točko
        if (sgn is None or not sgn) and  abs((df.iloc[i]['open']-df.iloc[prevPivot]["open"])/df.iloc[prevPivot]["open"]) > percent:
            sgnN = True
            break
        elif (sgn is None or sgn) and  abs((df.iloc[i]['open']-df.iloc[prevPivot]["open"])/df.iloc[prevPivot]["open"]) > percent:
            sgnN = False
            break
    if  i == len(df.index)-1:
            return [prevPivot]

    return [prevPivot] + trendReversals(df,percent,prevPivot,sgnN)


In [ ]:
if os.path.isfile('./ticker400_data.pickle'):
    with open('ticker400_data.pickle', 'rb') as handle:
        ticker400_data = pickle.load(handle)
    with open('ticker400_trend15.pickle', 'rb') as handle:
        ticker400_trend15 = pickle.load(handle)
    with open('ticker400_tren10.pickle', 'rb') as handle:
        ticker400_trend10 = pickle.load(handle)
    with open('ticker400_trend20.pickle', 'rb') as handle:
        ticker400_trend20 = pickle.load(handle)
else:

    ticker400_trend10 = {}
    ticker400_trend15 = {}
    ticker400_trend20 = {}
    for t in ticker400_data:
        ticker400_data[t] = ticker400_data[t].sort_values(by=['date'])
        ticker400_data[t]["return"] = (ticker400_data[t]["open"] - ticker400_data[t].shift(1,fill_value=ticker400_data[t]['open'].iloc[0])["open"])/ticker400_data[t].shift(1, fill_value=ticker400_data[t]['open'].iloc[0])["open"]
        ticker400_data[t]["volume%"] =  (ticker400_data[t]["volume"] - ticker400_data[t].shift(1,fill_value=ticker400_data[t]['volume'].iloc[0])["volume"])/ticker400_data[t].shift(1, fill_value=ticker400_data[t]['volume'].iloc[0])["volume"]
        #delamo na vrsticah >n-1
        ticker400_data[t]["volume5%"] = (ticker400_data[t]["volume"] - ticker400_data[t]['volume'].rolling(5).mean())/ticker400_data[t]['volume'].rolling(5).mean()
        ticker400_data[t]["volume10%"] = (ticker400_data[t]["volume"] - ticker400_data[t]['volume'].rolling(10).mean())/ticker400_data[t]['volume'].rolling(10).mean()
        ticker400_data[t]["volume15%"] = (ticker400_data[t]["volume"] - ticker400_data[t]['volume'].rolling(15).mean())/ticker400_data[t]['volume'].rolling(15).mean()
        ticker400_data[t]["volume20%"] = (ticker400_data[t]["volume"] - ticker400_data[t]['volume'].rolling(20).mean())/ticker400_data[t]['volume'].rolling(20).mean()
        ticker400_trend15[t] = trendReversals(ticker400_data[t],0.15)
        ticker400_trend20[t] = trendReversals(ticker400_data[t],0.20)
        ticker400_trend10[t] = trendReversals(ticker400_data[t],0.10)
        ticker400_data[t].reset_index(drop=True,inplace=True)


In [ ]:
ticker400_trend10B = {}
ticker400_trend15B = {}
ticker400_trend20B = {}

for t in ticker400_data:
    ticker400_trend10B[t] = pd.DataFrame(data=[False]*len(ticker400_data[t]))
    ticker400_trend10B[t].iloc[ticker400_trend10[t]] = True
    ticker400_trend15B[t] = pd.DataFrame(data=[False]*len(ticker400_data[t]))
    ticker400_trend15B[t].iloc[ticker400_trend15[t]] = True
    ticker400_trend20B[t] = pd.DataFrame(data=[False]*len(ticker400_data[t]))
    ticker400_trend20B[t].iloc[ticker400_trend20[t]] = True

In [ ]:
import pickle

with open('ticker400_data.pickle', 'wb') as handle:
    pickle.dump(ticker400_data, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('ticker400_tren10.pickle','wb') as fp:
    pickle.dump(ticker400_trend10,fp,protocol=pickle.HIGHEST_PROTOCOL)
with open('ticker400_trend15.pickle','wb') as fp1:
    pickle.dump(ticker400_trend15,fp1,protocol=pickle.HIGHEST_PROTOCOL)
with open('ticker400_trend20.pickle','wb') as fp2:
    pickle.dump(ticker400_trend20,fp2,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
splits25 = {g: train_test_split(ticker400_data[g],shuffle=False)for g in ticker400_data}
splits25_YB = {g:{10:train_test_split(ticker400_trend10B[g],shuffle=False),15:train_test_split(ticker400_trend15B[g],shuffle=False),20:train_test_split(ticker400_trend20B[g],shuffle=False)} for g in ticker400_data}
#splits50rolling = {g: train_test_split(ticker400_data[g], train_size=0.5,shuffle=False)for g in ticker400_data}
splits50rolling = {}

In [ ]:
def find_intersect_2_ordered_lists(list1,list2):
    li2=li1=0
    intersect = []
    if list1 is None or list2 is None or len(list1) == 0 or len(list2)==0 :
        return []
    while True:
        if list1[li1] == list2[li2]:
            intersect.append(list1[li1])
            li1 = li1 + 1
            li2 = li2 + 1
        elif list1[li1] < list2[li2]:
            li1 = li1 + 1
        else:
            li2 = li2 + 1
        if li1 == len(list1) or li2 == len(list2):
            return intersect

def all_true(truers):
    ret = True
    for j in truers:
        if not j:
            return False
    return True

def combine_lists(lists):
    idxs = [0 for l in lists]
    lens = [len(l) for l in lists]
    comb = []
    while True:
        if all_true([l == k for (l,k) in zip(idxs,lens)]):
            return comb
        cur = idxs.index(min(idxs))

In [ ]:
def fit_params_AUC_lagged_combinedOR(df,dfY,tick,percent,n_lagged = 20):
    def fita(solution, index):
        #24 genov
        #solution = [retUP,volUP,vol5UP,vol10UP,vol15UP,vol20UP,retDW,volDW,vol5DW,vol10DW,vol15DW,vol20DW,priorRUp,priorVUp,priorvol5UP,priorvol10UP,priorvol15UP,priorvol20UP,priorRDW,priorVDW,priorvol5DW,priorvol10DW,priorvol15DW,priorvol20DW]
        for ite in range(len(solution)):
            if ite<12 and solution[ite] <= -1:
                return -1
            elif ite>=12 and solution[ite] < 0:
                return -1
        solution = [x[0] if hasattr(x,'__len__') else x for x in solution]
        #print(solution)
        col = 'volume%'
        if n_lagged == 5:
            col = 'volume5%'
        if n_lagged == 10:
            col = 'volume10%'
        if n_lagged == 15:
            col = 'volume15%'
        if n_lagged == 20:
            col = 'volume20%'
        retFUp = solution[0]
        volFUp = solution[1]
        retFDown = solution[6]
        volFDown = solution[7]
        fit = 0
        l_UP = []
        l_DW = []
        buildingblocksUP = []
        buildingblocksDW = []
        test = df[tick][0][n_lagged-1:]
        up = test.loc[(test['return'].shift(math.floor(solution[12])) > solution[0]) |
                      (test['volume%'].shift(math.floor(solution[13])) > solution[1]) |
                       (test['volume5%'].shift(math.floor(solution[14])) > solution[2]) |
                        (test['volume10%'].shift(math.floor(solution[15])) > solution[3]) |
                         (test['volume15%'].shift(math.floor(solution[16])) > solution[4]) |
                          (test['volume20%'].shift(math.floor(solution[17])) > solution[5])
                        ].index
        down = test.loc[(test['return'].shift(math.floor(solution[18])) > solution[6]) |
        (test['volume%'].shift(math.floor(solution[19])) > solution[7]) |
        (test['volume5%'].shift(math.floor(solution[20])) > solution[8]) |
        (test['volume10%'].shift(math.floor(solution[21])) > solution[9]) |
        (test['volume15%'].shift(math.floor(solution[22])) > solution[10]) |
        (test['volume20%'].shift(math.floor(solution[23])) > solution[11])
                        ].index


        res= pd.DataFrame(data=[False]*len(df[tick][0][n_lagged-1:]))

        cut = 0
        for k in reversed(up):
            if k < len(df[tick][0][n_lagged-1:]):
                break
            else:
                cut = cut+1
        if cut > 0:
            up = up[:-cut]
        #down = list(find_intersect_2_ordered_lists(lfVDw,lfRDw))
        cut = 0
        for k in reversed(down):
            if k < len(df[tick][0][n_lagged-1:]):
                break
            else:
                cut = cut+1
        if cut > 0:
            down = down[:-cut]
        if up is not None :
            res.iloc[up] = True
        if down is not None :
            res.iloc[down] = True
        #print(roc_auc_score(dfY[tick][percent][0],res))
        return roc_auc_score(dfY[tick][percent][0][n_lagged-1:],res)
    return fita

In [ ]:
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
count = 0
pos = 0
if os.path.isfile('./eval2.pkl'):
    ga_evaluation = pd.read_pickle('./eval2.pkl')
    with open('pos2.pickle', 'rb') as handle:
        pos = pickle.load(handle)
else:
    ga_evaluation = pd.DataFrame(columns=['St. gen', 'St. starsev','Velikost pop','Izbira starsev','K Turnir','Tip križanja','Tip muaticje','Mutacija z zamenjavo','Ustavitveni pogoj','AUC','Trajanje'])

t = c.head(1).index.values[0]
num_generations = [100]
num_parents_mating = [0.05,0.1,0.25,0.5]
sol_per_pop = [10,20,40,80,160,320]
#[retUP,volUP,vol5UP,vol10UP,vol15UP,vol20UP,retDW,volDW,vol5DW,vol10DW,vol15DW,vol20DW,priorRUp,priorVUp,priorvol5UP,priorvol10UP,priorvol15UP,priorvol20UP,priorRDW,priorVDW,priorvol5DW,priorvol10DW,priorvol15DW,priorvol20DW]
num_genes = 24
#!!!!!
gene_type = [np.float16 for x in range(12)]+[np.uint8 for x in range(12)]
init_range_low = -1
init_range_high = 8
parent_selection = ['sss','rws','sus','rank','random','tournament']
#if tournament
k_tournament = [0.05,0.1,0.15,0.2,0.4,0.5,0.6,0.7]
crossover_tip = ['single_point','two_points','uniform','scattered']
mutation_type=['random','swap','inversion','scramble',None]
#if mutation type random
mutation_by_replacement = [True,False]
#!!!!!
gene_space = [{'low':-1,'high':8} for x in range(12)]+[[y for y in range(20)] for j in range(12)]
save_best_solutions = True
suppress_warnings = True
stop_criteria = ['saturate_5','saturate_7','saturate_9','saturate_12','saturate_15',]

comb = list(itertools.product(num_generations,num_parents_mating,sol_per_pop,parent_selection,k_tournament,crossover_tip,mutation_type,mutation_by_replacement,stop_criteria))

print(len(comb))
#testiran bo za 10
for a in comb:
    if count <= pos and pos != 0 :
        #if count % 100:
            #print("running" + str(count))
        count = count + 1
        continue
    start = datetime.datetime.now()

    if count%100 == 0:
        print(count)
    if(a[3] == 'tournament'):
        if(a[6] == 'random'):
            #print("hello")
            ins = pygad.GA(num_genes=24,gene_space=gene_space,gene_type=gene_type,init_range_low=init_range_low,init_range_high=init_range_high,save_best_solutions=True,suppress_warnings=suppress_warnings,num_generations=a[0],num_parents_mating=math.ceil(a[1]*a[2]),fitness_func=fit_params_AUC_lagged_combinedOR(splits25,splits25_YB,t,10),sol_per_pop=a[2],parent_selection_type=a[3],K_tournament=math.ceil(a[4]*(math.ceil(a[1]*a[2]))),crossover_type=a[5],mutation_type=a[6],mutation_by_replacement=a[7],stop_criteria=a[8])
            ins.run()
            end = datetime.datetime.now()
            ga_evaluation = ga_evaluation.append({'St. gen':a[0], 'St. starsev':math.ceil(a[1]*a[2]),'Velikost pop': a[2],'Izbira starsev':a[3],'K Turnir':math.ceil(a[4]*(math.ceil(a[1]*a[2]))),'Tip križanja':a[5],'Tip muaticje':a[6],'Mutacija z zamenjavo':a[7],'Ustavitveni pogoj':a[8],'AUC':ins.best_solutions_fitness[-1],'Trajanje':end-start},ignore_index=True)
        else:
            if(a[7]):
                ins = pygad.GA(num_genes=24,gene_space=gene_space,gene_type=gene_type,init_range_low=init_range_low,init_range_high=init_range_high,save_best_solutions=True,suppress_warnings=suppress_warnings,num_generations=a[0],num_parents_mating=math.ceil(a[1]*a[2]),fitness_func=fit_params_AUC_lagged_combinedOR(splits25,splits25_YB,t,10),sol_per_pop=a[2],parent_selection_type=a[3],K_tournament=math.ceil(a[4]*(math.ceil(a[1]*a[2]))),crossover_type=a[5],mutation_type=a[6],mutation_by_replacement=a[7],stop_criteria=a[8])
                ins.run()
                end = datetime.datetime.now()
                ga_evaluation = ga_evaluation.append({'St. gen':a[0], 'St. starsev':math.ceil(a[1]*a[2]),'Velikost pop': a[2],'Izbira starsev':a[3],'K Turnir':math.ceil(a[4]*(math.ceil(a[1]*a[2]))),'Tip križanja':a[5],'Tip muaticje':a[6],'Mutacija z zamenjavo':a[7],'Ustavitveni pogoj':a[8],'AUC':ins.best_solutions_fitness[-1],'Trajanje':end-start},ignore_index=True)
    else:
        if(a[4] == 0.05):
            if(a[6] == 'random'):
                ins = pygad.GA(num_genes=24,gene_space=gene_space,gene_type=gene_type,init_range_low=init_range_low,init_range_high=init_range_high,save_best_solutions=True,suppress_warnings=suppress_warnings,num_generations=a[0],num_parents_mating=math.ceil(a[1]*a[2]),fitness_func=fit_params_AUC_lagged_combinedOR(splits25,splits25_YB,t,10),sol_per_pop=a[2],parent_selection_type=a[3],K_tournament=math.ceil(a[4]*(math.ceil(a[1]*a[2]))),crossover_type=a[5],mutation_type=a[6],mutation_by_replacement=a[7],stop_criteria=a[8])
                ins.run()
                end = datetime.datetime.now()
                ga_evaluation = ga_evaluation.append({'St. gen':a[0], 'St. starsev':math.ceil(a[1]*a[2]),'Velikost pop': a[2],'Izbira starsev':a[3],'K Turnir':math.ceil(a[4]*(math.ceil(a[1]*a[2]))),'Tip križanja':a[5],'Tip muaticje':a[6],'Mutacija z zamenjavo':a[7],'Ustavitveni pogoj':a[8],'AUC':ins.best_solutions_fitness[-1],'Trajanje':end-start},ignore_index=True)
            else:
                if(a[7]):
                    ins = pygad.GA(num_genes=24,gene_space=gene_space,gene_type=gene_type,init_range_low=init_range_low,init_range_high=init_range_high,save_best_solutions=True,suppress_warnings=suppress_warnings,num_generations=a[0],num_parents_mating=math.ceil(a[1]*a[2]),fitness_func=fit_params_AUC_lagged_combinedOR(splits25,splits25_YB,t,10),sol_per_pop=a[2],parent_selection_type=a[3],K_tournament=math.ceil(a[4]*(math.ceil(a[1]*a[2]))),crossover_type=a[5],mutation_type=a[6],mutation_by_replacement=a[7],stop_criteria=a[8])
                    ins.run()
                    end = datetime.datetime.now()
                    ga_evaluation = ga_evaluation.append({'St. gen':a[0], 'St. starsev':math.ceil(a[1]*a[2]),'Velikost pop': a[2],'Izbira starsev':a[3],'K Turnir':math.ceil(a[4]*(math.ceil(a[1]*a[2]))),'Tip križanja':a[5],'Tip muaticje':a[6],'Mutacija z zamenjavo':a[7],'Ustavitveni pogoj':a[8],'AUC':ins.best_solutions_fitness[-1],'Trajanje':end-start},ignore_index=True)
    count = count+1
    gene_type = [np.float16 for x in range(12)]+[np.uint8 for x in range(12)]
    if count % 100 == 0 and ins is not None:
        pd.to_pickle(ga_evaluation,'./eval2.pkl')
        with open('pos2.pickle', 'wb') as handle:
            pickle.dump(count, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(count)
pd.to_pickle(ga_evaluation,'./eval2.pkl')